In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from tqdm import tqdm_notebook
# Census API Key
from config import api_key
c = Census(api_key, year=2017)

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",'B15003_002E', 'B15003_003E', 
                          'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E',
                          'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E',
                          'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E', 'B15003_018E',
                          'B15003_019E', 'B15003_020E', "B15003_021E", "B15003_022E", "B15003_023E",
                          "B15003_024E", "B15003_025E", 'B15003_001E', 'B12001_001E', 'B12001_003E',
                          'B12001_004E', 'B12001_007E', 'B12001_009E', 'B12001_010E', 'B12001_011E', 
                          'B12001_012E', 'B12001_013E', 'B12001_016E', 'B12001_018E', 'B12001_019E')
                          , {'for': 'county:*'})


# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column Reordering
census_df = census_df.rename(columns={'B01003_001E': 'Population',
                                      'B01002_001E': 'Median Age',
                                      'B19013_001E': 'Household Income',
                                      'B19301_001E': 'Per Capita Income',
                                      'B17001_002E': 'Poverty Count',
                                      'NAME': 'Name', 
                                      'county': 'County',
                                      'B15003_021E' : 'Associate\'s Degree Count',
                                      'B15003_022E' : 'Bachelor\'s Degree Count',
                                      'B15003_023E' : 'Master\'s Degree Count',
                                      'B15003_024E' : 'Professional School Degree Count',
                                      'B15003_025E' : 'Doctorate Degree Count',
                                      'B15003_001E' : 'Total educational attainment over 25',
                                      'B15003_002E' : 'No School',
                                      'B15003_003E' : 'Nursery school',
                                      'B15003_004E' : 'Kindergarten',
                                      'B15003_005E' : '1st grade',
                                      'B15003_006E' : '2nd grade',
                                      'B15003_007E' : '3rd grade',
                                      'B15003_008E' : '4th grade',
                                      'B15003_009E' : '5th grade',
                                      'B15003_010E' : '6th grade',
                                      'B15003_011E' : '7th grade',
                                      'B15003_012E' : '8th grade',
                                      'B15003_013E' : '9th grade',
                                      'B15003_014E' : '10th grade',
                                      'B15003_015E' : '11th grade',
                                      'B15003_016E' : '12th grade No Diploma',
                                      'B15003_017E' : 'HS Diploma',
                                      'B15003_018E' : 'GED',
                                      'B15003_019E' : 'Some College less than 1 year',
                                      'B15003_020E' : 'Some College More than 1 year',
                                      'B12001_001E' : 'Over 15 Marital Status',
                                      'B12001_003E' : 'Male Never Married',
                                      'B12001_004E' : 'Male Married',
                                      'B12001_007E' : 'Male Spouse Seperated',
                                      'B12001_009E' : 'Male Widowed',
                                      'B12001_010E' : 'Male Divorced',
                                      'B12001_012E' : 'Female Never Married',
                                      'B12001_013E' : 'Female Married',
                                      'B12001_016E' : 'Female Spouse Seperated',
                                      'B12001_018E' : 'Female Widowed',
                                      'B12001_019E' : 'Female Divorced'})

census_df["Poverty Rate"] = (100 * census_df["Poverty Count"].astype(int)
                            / census_df["Population"].astype(int))
census_df["Total College Degrees"] = (census_df['Associate\'s Degree Count'].astype(int)+
                             census_df['Bachelor\'s Degree Count'].astype(int)+
                             census_df['Master\'s Degree Count'].astype(int)+
                             census_df['Professional School Degree Count'].astype(int)+
                             census_df['Doctorate Degree Count'].astype(int))
census_df['Total No HS Diploma'] = (census_df['No School'].astype(int)+
                                    census_df['Nursery school'].astype(int)+
                                    census_df['Kindergarten'].astype(int)+
                                    census_df['1st grade'].astype(int)+
                                    census_df['2nd grade'].astype(int)+
                                    census_df['3rd grade'].astype(int)+
                                    census_df['4th grade'].astype(int)+
                                    census_df['5th grade'].astype(int)+
                                    census_df['6th grade'].astype(int)+
                                    census_df['7th grade'].astype(int)+
                                    census_df['8th grade'].astype(int)+
                                    census_df['9th grade'].astype(int)+
                                    census_df['10th grade'].astype(int)+
                                    census_df['11th grade'].astype(int)+
                                    census_df['12th grade No Diploma'].astype(int))
census_df['Total HS Diploma'] = (census_df['HS Diploma'].astype(int)+
                                    census_df['GED'].astype(int))

census_df['Total Some College'] = (census_df['Some College less than 1 year'].astype(int)+
                                    census_df['Some College More than 1 year'].astype(int))

census_df['Education Check'] = (census_df['Total College Degrees'].astype(int)+
                             census_df['Total No HS Diploma'].astype(int)+
                             census_df['Total HS Diploma'].astype(int)+
                             census_df['Total Some College'].astype(int))

census_df["Never Married"] = (census_df['Male Never Married'].astype(int)+
                             census_df['Female Never Married'].astype(int))


census_df["Married"] = (census_df['Male Married'].astype(int)+
                        census_df['Female Married'].astype(int))
                        
census_df["Widowed"] = (census_df['Male Widowed'].astype(int)+
                             census_df['Female Widowed'].astype(int))

census_df["Divorced"] = (census_df['Male Divorced'].astype(int)+
                             census_df['Female Divorced'].astype(int))

census_df['Married Check'] = (census_df['Married'].astype(int)+
                             census_df['Widowed'].astype(int)+
                             census_df['Divorced'].astype(int)+
                             census_df["Never Married"].astype(int))

census_df['College Degree Rate'] = (census_df['Total College Degrees']/census_df['Total educational attainment over 25']*100)

census_df['No High School Diploma Rate'] = (census_df['Total No HS Diploma']/census_df['Total educational attainment over 25']*100)

census_df['HS Diploma Rate'] = (census_df['Total HS Diploma']/census_df['Total educational attainment over 25']*100)

census_df['Some College Rate'] = (census_df['Total Some College']/census_df['Total educational attainment over 25']*100)

census_df['Married Rate'] = (census_df['Married']/census_df['Over 15 Marital Status']*100)

census_df['Widowed Rate'] = (census_df['Widowed']/census_df['Over 15 Marital Status']*100)

census_df['Divorced Rate'] = (census_df['Divorced']/census_df['Over 15 Marital Status']*100)

census_df['Never Married Rate'] = (census_df['Never Married']/census_df['Over 15 Marital Status']*100)

# Final DataFrame
census_df = census_df[['Name', 'Population', 'Median Age', 'Household Income',
                       'Per Capita Income', 'Poverty Count', 'Poverty Rate', 'Total No HS Diploma', 'No High School Diploma Rate',
                       'Total HS Diploma', 'HS Diploma Rate', 'Total Some College', 'Some College Rate', 'Total College Degrees',
                       'College Degree Rate', 'Total educational attainment over 25','Never Married', 'Never Married Rate', 'Married','Married Rate', 'Widowed',
                       'Widowed Rate', 'Divorced', 'Divorced Rate', 'Over 15 Marital Status']]

# Visualize
print(len(census_df))

3220


In [3]:
census_df.loc[census_df['Household Income'] < 0, 'Household Income'] = 0
census_df.loc[census_df['Median Age'] < 0, 'Median Age'] = 0
census_df.loc[census_df['Per Capita Income'] < 0, 'Per Capita Income'] = 0

In [4]:
census_df=census_df.sort_values(by='Name', ascending=True)
census_df = census_df.reset_index(drop=True)
State_County = census_df["Name"].str.split(", ", n = 1, expand = True)

In [5]:
State_County = State_County.rename(columns={0: "County", 1: "State"})

In [6]:
census_df = State_County.join(census_df, how='outer')
census_df = census_df.drop(columns=['Name'])
census_df.head()

,County,State,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Total No HS Diploma,No High School Diploma Rate,...,Total educational attainment over 25,Never Married,Never Married Rate,Married,Married Rate,Widowed,Widowed Rate,Divorced,Divorced Rate,Over 15 Marital Status
0,Abbeville County,South Carolina,24788.0,43.7,35254.0,19234.0,5474.0,22.083266,3687,21.548802,...,17110.0,6396,31.207612,9887,48.241034,2010,9.807270,2202,10.744084,20495.0
1,Acadia Parish,Louisiana,62607.0,35.8,40492.0,21591.0,13196.0,21.077515,8984,22.352151,...,40193.0,14969,30.768756,24605,50.575540,3280,6.742035,5796,11.913669,48650.0
2,Accomack County,Virginia,32840.0,46.0,42260.0,24266.0,6416.0,19.537150,4199,17.824094,...,23558.0,7003,25.853731,15458,57.067966,2285,8.435781,2341,8.642522,27087.0
3,Ada County,Idaho,435117.0,36.4,60151.0,31642.0,50474.0,11.600098,15350,5.304554,...,289374.0,104669,30.255266,184610,53.362740,14724,4.256069,41950,12.125925,345953.0
4,Adair County,Iowa,7192.0,45.7,49477.0,28861.0,661.0,9.190768,309,5.980259,...,5167.0,1243,21.007267,3598,60.807842,477,8.061518,599,10.123373,5917.0


In [7]:
census_df = census_df.loc[census_df['State'] != 'Puerto Rico']

In [8]:
census_df.loc[census_df['State'] == 'Puerto Rico']

,County,State,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Total No HS Diploma,No High School Diploma Rate,...,Total educational attainment over 25,Never Married,Never Married Rate,Married,Married Rate,Widowed,Widowed Rate,Divorced,Divorced Rate,Over 15 Marital Status


In [9]:
census_df.to_csv('Project1_County_Data.csv',encoding="utf-8", index=False)